# Working with data

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training Data
training_data = datasets.FashionMNIST(root="data", train=True, download=True,transform=ToTensor())

test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [4]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [BS, C, H, W]: {X.shape}")
    print(f"Shape of y : {y.shape}, {y.dtype}")
    break

Shape of X [BS, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]), torch.int64


# Creating Models

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

Using cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Optimizing the Model Paramters

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction & error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [8]:
@torch.no_grad()
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            # Compute prediction & error
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Acc: {(100 * correct):>0.1f}%, Avg Loss : {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epochs {t+1}\n-------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done")

Epochs 1
-------------------------------------
loss: 2.299059  [    0/60000]
loss: 2.285537  [ 6400/60000]
loss: 2.264471  [12800/60000]
loss: 2.260520  [19200/60000]
loss: 2.247567  [25600/60000]
loss: 2.208173  [32000/60000]
loss: 2.216513  [38400/60000]
loss: 2.178077  [44800/60000]
loss: 2.179294  [51200/60000]
loss: 2.153411  [57600/60000]
Test Error: 
 Acc: 45.8%, Avg Loss : 2.145004 

Epochs 2
-------------------------------------
loss: 2.147297  [    0/60000]
loss: 2.138712  [ 6400/60000]
loss: 2.079991  [12800/60000]
loss: 2.108046  [19200/60000]
loss: 2.057697  [25600/60000]
loss: 1.986574  [32000/60000]
loss: 2.012696  [38400/60000]
loss: 1.928171  [44800/60000]
loss: 1.936637  [51200/60000]
loss: 1.867384  [57600/60000]
Test Error: 
 Acc: 57.9%, Avg Loss : 1.864523 

Epochs 3
-------------------------------------
loss: 1.885907  [    0/60000]
loss: 1.856161  [ 6400/60000]
loss: 1.741849  [12800/60000]
loss: 1.798444  [19200/60000]
loss: 1.684978  [25600/60000]
loss: 1.63037

# Saving Models

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Loading Models

In [11]:
model_ = NeuralNetwork()
model_.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model_.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred= model_(x)
    print(pred.shape)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


torch.Size([1, 10])
Predicted: "Ankle boot", Actual: "Ankle boot"
